In [1]:
import numpy as np
from sklearn.metrics import normalized_mutual_info_score

import torch
import torch.nn.functional as F
import torch_geometric as tg
from torch_geometric.nn import DenseSAGEConv, dense_diff_pool, dense_mincut_pool
from torch_geometric.utils import to_dense_adj

from dataset import load_pyg_dataset

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('cuda', 1)


import os
import pandas as pd
import os.path as osp

In [3]:
def save_best_model(model, optimizer, epoch, outdir):
    torch.save(
        {
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
        },
        os.path.join(outdir,
                     'best_model.pt'))

def load_best_model(model, outdir):
    checkpoint = torch.load(os.path.join(outdir, 'best_model.pt'))
    model.load_state_dict(checkpoint['model_state_dict'])
    return model

In [4]:
pooling = 'mincut' #(options: diffpool, mincut)
# dataset_name = 'Chameleon'



#Pooling selector
pooling_selector = {
    'diffpool': dense_diff_pool,
    'mincut': dense_mincut_pool
}




# 'WikiCS',
for dataset_name in ['Cornell', 'Texas', 'Wisconsin', "Chameleon", "Squirrel", "Crocodile", 'Actor',
                    'DeezerEurope', 'Cora', 'Pubmed', 'Citeseer', 
                    'Twitch-DE', 'Twitch-EN', 'Twitch-ES', 'Twitch-FR', 'Twitch-PT', 'Twitch-RU']:

    #Creates outdir
    outdir = 'results_node_clustering/{}_{}'.format(dataset_name, pooling)
    os.makedirs(outdir, exist_ok = True)

    #Loads dataset
    data = load_pyg_dataset(
        data_name=dataset_name,
        device=device
    )
    num_clusters = data.y.max().tolist()+1
    data.adj = to_dense_adj(data.edge_index)


    #GNN to compute transformed node features for pooling (for assignation matrix)
    class GNN(torch.nn.Module):
        def __init__(self, in_channels, hidden_channels, out_channels,
                     normalize=False, lin=True):
            super(GNN, self).__init__()
            self.conv1 = DenseSAGEConv(in_channels, hidden_channels, normalize)
            self.bn1 = torch.nn.BatchNorm1d(hidden_channels)
            self.conv2 = DenseSAGEConv(hidden_channels, hidden_channels, normalize)
            self.bn2 = torch.nn.BatchNorm1d(hidden_channels)
            self.conv3 = DenseSAGEConv(hidden_channels, out_channels, normalize)
            self.bn3 = torch.nn.BatchNorm1d(out_channels)
            if lin is True:
                self.lin = torch.nn.Linear(
                    2 * hidden_channels + out_channels,
                    out_channels
                )
            else:
                self.lin = None

        def bn(self, i, x):
            batch_size, num_nodes, num_channels = x.size()
            x = x.view(-1, num_channels)
            x = getattr(self, 'bn{}'.format(i))(x)
            x = x.view(batch_size, num_nodes, num_channels)
            return x

        def forward(self, x, adj, mask=None):
            x0 = x
            if pooling == 'diffpool':
                x1 = self.bn(1, F.relu(self.conv1(x0, adj, mask)))
                x2 = self.bn(2, F.relu(self.conv2(x1, adj, mask)))
                x3 = self.bn(3, F.relu(self.conv3(x2, adj, mask)))
            elif pooling == 'mincut':
                x1 = F.relu(self.conv1(x0, adj, mask))
                x2 = F.relu(self.conv2(x1, adj, mask))
                x3 = F.relu(self.conv3(x2, adj, mask))
            x = torch.cat([x1, x2, x3], dim=-1)
            if self.lin is not None:
                x = F.relu(self.lin(x))
            return x

    #Net to compute pooling in an unsupervised manner
    class Net(torch.nn.Module):
        def __init__(self):
            super(Net, self).__init__()
            self.gnn1_pool = GNN(data.num_node_features, 64, num_clusters)
            self.gnn1_embed = GNN(data.num_node_features, 64, 64, lin=False)
            self.pooling = pooling_selector[pooling]

        def forward(self, x, adj, mask=None):
            s = self.gnn1_pool(x, adj, mask)
            x = self.gnn1_embed(x, adj, mask)
            x, adj, l1, e1 = self.pooling(x, adj, s, mask)
            return torch.softmax(s, dim=-1), l1, e1, adj # returns assignation matrix, and auxiliary losses and new adj matrix




    ###########
    #Training
    ###########
    #Optimizer, model
    model = Net().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    #Trains using auxiliary losses only
    def train(epoch):
        model.train()
        optimizer.zero_grad()
        s, l1, e1, adj = model(data.x, data.adj)
        loss = l1 + e1
        loss.backward()
        optimizer.step()
        return loss

    #Measures mutual information of clustering computed by pooling and ground truth
    @torch.no_grad()
    def test():
        model.eval()
        pred_node_label = model(data.x, data.adj)[0].max(dim=-1)[1].detach().cpu().numpy() 
        truth_node_labels = data.y.cpu().numpy()
        nmi = normalized_mutual_info_score(truth_node_labels.flatten(), pred_node_label.flatten())
        return nmi


    log_handle = open(osp.join(outdir, 'log.txt'), 'w')
    best_val_acc = best_test_acc = 0
    for epoch in range(1, 251):
        train_loss = train(epoch)
        val_acc = test()
        test_acc = test()
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_test_acc = test_acc
            save_best_model(model, optimizer, epoch, outdir)
        print(f'Epoch: {epoch:03d}, Train Loss: {train_loss:.8f}, '
              f'Val NMI: {val_acc:.4f}, Test NMI: {test_acc:.4f}')
        log_handle.write(f'Epoch: {epoch:03d}, Train Loss: {train_loss:.8f}, '
              f'Val NMI: {val_acc:.4f}, Test NMI: {test_acc:.4f}\n')

    print(f'Best Val NMI: {best_val_acc:.4f}, Test NMI: {best_test_acc:.4f}')
    log_handle.write(f'\nBest Val NMI: {best_val_acc:.4f}, Test NMI: {best_test_acc:.4f}\n')
    log_handle.close()



    #Loads best model
    model = load_best_model(model, outdir)
    model.eval()
    node_assignation = model(data.x, data.adj)[0].max(-1)[1].detach().cpu().numpy().squeeze()
    adj_matrix = model(data.x, data.adj)[3].detach().cpu().numpy().squeeze()


    #Computes node clustering and adj matrices and saves them
    pd.DataFrame(node_assignation).transpose().to_csv(osp.join(outdir, 'node_assignation.csv'), header = None, index=False)
    pd.DataFrame(adj_matrix).to_csv(osp.join(outdir, 'adj_matrix.csv'), header = None, index=False)



The obtained data Cornell has 183 nodes, 298 edges, 1703 features, 5 labels, 
Epoch: 001, Train Loss: 0.05161363, Val NMI: 0.0299, Test NMI: 0.0299
Epoch: 002, Train Loss: 0.05140239, Val NMI: 0.0366, Test NMI: 0.0366
Epoch: 003, Train Loss: 0.05127370, Val NMI: 0.0558, Test NMI: 0.0558
Epoch: 004, Train Loss: 0.05092454, Val NMI: 0.0434, Test NMI: 0.0434
Epoch: 005, Train Loss: 0.05018914, Val NMI: 0.0403, Test NMI: 0.0403
Epoch: 006, Train Loss: 0.04891694, Val NMI: 0.0275, Test NMI: 0.0275
Epoch: 007, Train Loss: 0.04680693, Val NMI: 0.0421, Test NMI: 0.0421
Epoch: 008, Train Loss: 0.04348505, Val NMI: 0.0555, Test NMI: 0.0555
Epoch: 009, Train Loss: 0.03848708, Val NMI: 0.0555, Test NMI: 0.0555
Epoch: 010, Train Loss: 0.03134322, Val NMI: 0.0476, Test NMI: 0.0476
Epoch: 011, Train Loss: 0.02173781, Val NMI: 0.0298, Test NMI: 0.0298
Epoch: 012, Train Loss: 0.00983655, Val NMI: 0.0342, Test NMI: 0.0342
Epoch: 013, Train Loss: -0.00351650, Val NMI: 0.0207, Test NMI: 0.0207
Epoch: 014,

Epoch: 122, Train Loss: -0.34678781, Val NMI: 0.0111, Test NMI: 0.0111
Epoch: 123, Train Loss: -0.34679061, Val NMI: 0.0111, Test NMI: 0.0111
Epoch: 124, Train Loss: -0.34678978, Val NMI: 0.0111, Test NMI: 0.0111
Epoch: 125, Train Loss: -0.34679317, Val NMI: 0.0111, Test NMI: 0.0111
Epoch: 126, Train Loss: -0.34680176, Val NMI: 0.0111, Test NMI: 0.0111
Epoch: 127, Train Loss: -0.34681082, Val NMI: 0.0111, Test NMI: 0.0111
Epoch: 128, Train Loss: -0.34681571, Val NMI: 0.0111, Test NMI: 0.0111
Epoch: 129, Train Loss: -0.34681803, Val NMI: 0.0111, Test NMI: 0.0111
Epoch: 130, Train Loss: -0.34682095, Val NMI: 0.0111, Test NMI: 0.0111
Epoch: 131, Train Loss: -0.34682626, Val NMI: 0.0111, Test NMI: 0.0111
Epoch: 132, Train Loss: -0.34683335, Val NMI: 0.0111, Test NMI: 0.0111
Epoch: 133, Train Loss: -0.34683979, Val NMI: 0.0111, Test NMI: 0.0111
Epoch: 134, Train Loss: -0.34684461, Val NMI: 0.0111, Test NMI: 0.0111
Epoch: 135, Train Loss: -0.34684807, Val NMI: 0.0111, Test NMI: 0.0111
Epoch:

Epoch: 245, Train Loss: -0.34701037, Val NMI: 0.0111, Test NMI: 0.0111
Epoch: 246, Train Loss: -0.34694153, Val NMI: 0.0111, Test NMI: 0.0111
Epoch: 247, Train Loss: -0.34686762, Val NMI: 0.0111, Test NMI: 0.0111
Epoch: 248, Train Loss: -0.34685045, Val NMI: 0.0111, Test NMI: 0.0111
Epoch: 249, Train Loss: -0.34688735, Val NMI: 0.0111, Test NMI: 0.0111
Epoch: 250, Train Loss: -0.34695959, Val NMI: 0.0111, Test NMI: 0.0111
Best Val NMI: 0.0558, Test NMI: 0.0558
The obtained data Texas has 183 nodes, 325 edges, 1703 features, 5 labels, 
Epoch: 001, Train Loss: 0.05180478, Val NMI: 0.0829, Test NMI: 0.0829
Epoch: 002, Train Loss: 0.05117494, Val NMI: 0.0710, Test NMI: 0.0710
Epoch: 003, Train Loss: 0.04974759, Val NMI: 0.0431, Test NMI: 0.0431
Epoch: 004, Train Loss: 0.04594672, Val NMI: 0.0155, Test NMI: 0.0155
Epoch: 005, Train Loss: 0.03834820, Val NMI: 0.0000, Test NMI: 0.0000
Epoch: 006, Train Loss: 0.02595472, Val NMI: 0.0000, Test NMI: 0.0000
Epoch: 007, Train Loss: 0.01223135, Val

Epoch: 114, Train Loss: -0.29215872, Val NMI: 0.0288, Test NMI: 0.0288
Epoch: 115, Train Loss: -0.29244435, Val NMI: 0.0288, Test NMI: 0.0288
Epoch: 116, Train Loss: -0.29270709, Val NMI: 0.0288, Test NMI: 0.0288
Epoch: 117, Train Loss: -0.29278445, Val NMI: 0.0288, Test NMI: 0.0288
Epoch: 118, Train Loss: -0.29267061, Val NMI: 0.0288, Test NMI: 0.0288
Epoch: 119, Train Loss: -0.29252005, Val NMI: 0.0288, Test NMI: 0.0288
Epoch: 120, Train Loss: -0.29253769, Val NMI: 0.0288, Test NMI: 0.0288
Epoch: 121, Train Loss: -0.29268169, Val NMI: 0.0288, Test NMI: 0.0288
Epoch: 122, Train Loss: -0.29284036, Val NMI: 0.0288, Test NMI: 0.0288
Epoch: 123, Train Loss: -0.29286766, Val NMI: 0.0288, Test NMI: 0.0288
Epoch: 124, Train Loss: -0.29278529, Val NMI: 0.0288, Test NMI: 0.0288
Epoch: 125, Train Loss: -0.29270792, Val NMI: 0.0288, Test NMI: 0.0288
Epoch: 126, Train Loss: -0.29272437, Val NMI: 0.0288, Test NMI: 0.0288
Epoch: 127, Train Loss: -0.29284120, Val NMI: 0.0288, Test NMI: 0.0288
Epoch:

Epoch: 236, Train Loss: -0.29845941, Val NMI: 0.0337, Test NMI: 0.0337
Epoch: 237, Train Loss: -0.29844731, Val NMI: 0.0337, Test NMI: 0.0337
Epoch: 238, Train Loss: -0.29845792, Val NMI: 0.0337, Test NMI: 0.0337
Epoch: 239, Train Loss: -0.29848224, Val NMI: 0.0337, Test NMI: 0.0337
Epoch: 240, Train Loss: -0.29850030, Val NMI: 0.0337, Test NMI: 0.0337
Epoch: 241, Train Loss: -0.29850155, Val NMI: 0.0337, Test NMI: 0.0337
Epoch: 242, Train Loss: -0.29849094, Val NMI: 0.0337, Test NMI: 0.0337
Epoch: 243, Train Loss: -0.29847962, Val NMI: 0.0337, Test NMI: 0.0337
Epoch: 244, Train Loss: -0.29847765, Val NMI: 0.0337, Test NMI: 0.0337
Epoch: 245, Train Loss: -0.29848480, Val NMI: 0.0337, Test NMI: 0.0337
Epoch: 246, Train Loss: -0.29849726, Val NMI: 0.0337, Test NMI: 0.0337
Epoch: 247, Train Loss: -0.29850686, Val NMI: 0.0337, Test NMI: 0.0337
Epoch: 248, Train Loss: -0.29851043, Val NMI: 0.0337, Test NMI: 0.0337
Epoch: 249, Train Loss: -0.29850763, Val NMI: 0.0337, Test NMI: 0.0337
Epoch:

...

Epoch: 038, Train Loss: -0.68197286, Val NMI: 0.0199, Test NMI: 0.0199
Epoch: 039, Train Loss: -0.68588877, Val NMI: 0.0199, Test NMI: 0.0199
Epoch: 040, Train Loss: -0.68947530, Val NMI: 0.0195, Test NMI: 0.0195
Epoch: 041, Train Loss: -0.69272047, Val NMI: 0.0210, Test NMI: 0.0210
Epoch: 042, Train Loss: -0.69567525, Val NMI: 0.0212, Test NMI: 0.0212
Epoch: 043, Train Loss: -0.69839567, Val NMI: 0.0202, Test NMI: 0.0202
Epoch: 044, Train Loss: -0.70085442, Val NMI: 0.0201, Test NMI: 0.0201
Epoch: 045, Train Loss: -0.70313561, Val NMI: 0.0206, Test NMI: 0.0206
Epoch: 046, Train Loss: -0.70521832, Val NMI: 0.0206, Test NMI: 0.0206
Epoch: 047, Train Loss: -0.70720857, Val NMI: 0.0201, Test NMI: 0.0201
Epoch: 048, Train Loss: -0.70906937, Val NMI: 0.0195, Test NMI: 0.0195
Epoch: 049, Train Loss: -0.71086121, Val NMI: 0.0195, Test NMI: 0.0195
Epoch: 050, Train Loss: -0.71253973, Val NMI: 0.0197, Test NMI: 0.0197
Epoch: 051, Train Loss: -0.71414518, Val NMI: 0.0201, Test NMI: 0.0201
Epoch:

Epoch: 154, Train Loss: -0.74359226, Val NMI: 0.0181, Test NMI: 0.0181
Epoch: 155, Train Loss: -0.74363250, Val NMI: 0.0188, Test NMI: 0.0188
Epoch: 156, Train Loss: -0.74369085, Val NMI: 0.0190, Test NMI: 0.0190
Epoch: 157, Train Loss: -0.74375570, Val NMI: 0.0190, Test NMI: 0.0190
Epoch: 158, Train Loss: -0.74380571, Val NMI: 0.0190, Test NMI: 0.0190
Epoch: 159, Train Loss: -0.74384391, Val NMI: 0.0187, Test NMI: 0.0187
Epoch: 160, Train Loss: -0.74388659, Val NMI: 0.0190, Test NMI: 0.0190
Epoch: 161, Train Loss: -0.74393314, Val NMI: 0.0190, Test NMI: 0.0190
Epoch: 162, Train Loss: -0.74397182, Val NMI: 0.0187, Test NMI: 0.0187
Epoch: 163, Train Loss: -0.74400097, Val NMI: 0.0190, Test NMI: 0.0190
Epoch: 164, Train Loss: -0.74403083, Val NMI: 0.0187, Test NMI: 0.0187
Epoch: 165, Train Loss: -0.74406606, Val NMI: 0.0185, Test NMI: 0.0185
Epoch: 166, Train Loss: -0.74410266, Val NMI: 0.0185, Test NMI: 0.0185
Epoch: 167, Train Loss: -0.74413359, Val NMI: 0.0185, Test NMI: 0.0185
Epoch:

Epoch: 017, Train Loss: -0.36635727, Val NMI: 0.0009, Test NMI: 0.0009
Epoch: 018, Train Loss: -0.38593990, Val NMI: 0.0010, Test NMI: 0.0010
Epoch: 019, Train Loss: -0.40652299, Val NMI: 0.0011, Test NMI: 0.0011
Epoch: 020, Train Loss: -0.42786565, Val NMI: 0.0011, Test NMI: 0.0011
Epoch: 021, Train Loss: -0.44961154, Val NMI: 0.0013, Test NMI: 0.0013
Epoch: 022, Train Loss: -0.47134179, Val NMI: 0.0016, Test NMI: 0.0016
Epoch: 023, Train Loss: -0.49267855, Val NMI: 0.0015, Test NMI: 0.0015
Epoch: 024, Train Loss: -0.51321715, Val NMI: 0.0018, Test NMI: 0.0018
Epoch: 025, Train Loss: -0.53269666, Val NMI: 0.0015, Test NMI: 0.0015
Epoch: 026, Train Loss: -0.55087316, Val NMI: 0.0015, Test NMI: 0.0015
Epoch: 027, Train Loss: -0.56762177, Val NMI: 0.0012, Test NMI: 0.0012
Epoch: 028, Train Loss: -0.58290184, Val NMI: 0.0012, Test NMI: 0.0012
Epoch: 029, Train Loss: -0.59676254, Val NMI: 0.0014, Test NMI: 0.0014
Epoch: 030, Train Loss: -0.60928553, Val NMI: 0.0015, Test NMI: 0.0015
Epoch:

Epoch: 133, Train Loss: -0.75432152, Val NMI: 0.0007, Test NMI: 0.0007
Epoch: 134, Train Loss: -0.75521076, Val NMI: 0.0007, Test NMI: 0.0007
Epoch: 135, Train Loss: -0.75481528, Val NMI: 0.0006, Test NMI: 0.0006
Epoch: 136, Train Loss: -0.75451887, Val NMI: 0.0007, Test NMI: 0.0007
Epoch: 137, Train Loss: -0.75544006, Val NMI: 0.0007, Test NMI: 0.0007
Epoch: 138, Train Loss: -0.75479412, Val NMI: 0.0006, Test NMI: 0.0006
Epoch: 139, Train Loss: -0.75467461, Val NMI: 0.0007, Test NMI: 0.0007
Epoch: 140, Train Loss: -0.75561035, Val NMI: 0.0007, Test NMI: 0.0007
Epoch: 141, Train Loss: -0.75463039, Val NMI: 0.0006, Test NMI: 0.0006
Epoch: 142, Train Loss: -0.75493336, Val NMI: 0.0006, Test NMI: 0.0006
Epoch: 143, Train Loss: -0.75565058, Val NMI: 0.0007, Test NMI: 0.0007
Epoch: 144, Train Loss: -0.75455296, Val NMI: 0.0006, Test NMI: 0.0006
Epoch: 145, Train Loss: -0.75542569, Val NMI: 0.0006, Test NMI: 0.0006
Epoch: 146, Train Loss: -0.75559199, Val NMI: 0.0006, Test NMI: 0.0006
Epoch:

Epoch: 249, Train Loss: -0.75890124, Val NMI: 0.0006, Test NMI: 0.0006
Epoch: 250, Train Loss: -0.75892311, Val NMI: 0.0006, Test NMI: 0.0006
Best Val NMI: 0.0018, Test NMI: 0.0018


In [5]:
pooling = 'mincut' #(options: diffpool, mincut)
# dataset_name = 'Chameleon'



#Pooling selector
pooling_selector = {
    'diffpool': dense_diff_pool,
    'mincut': dense_mincut_pool
}




# 'WikiCS',
for dataset_name in ['WikiCS']:

    #Creates outdir
    outdir = 'results_node_clustering/{}_{}'.format(dataset_name, pooling)
    os.makedirs(outdir, exist_ok = True)

    #Loads dataset
    data = load_pyg_dataset(
        data_name=dataset_name,
        device=device
    )
    num_clusters = data.y.max().tolist()+1
    data.adj = to_dense_adj(data.edge_index)


    #GNN to compute transformed node features for pooling (for assignation matrix)
    class GNN(torch.nn.Module):
        def __init__(self, in_channels, hidden_channels, out_channels,
                     normalize=False, lin=True):
            super(GNN, self).__init__()
            self.conv1 = DenseSAGEConv(in_channels, hidden_channels, normalize)
            self.bn1 = torch.nn.BatchNorm1d(hidden_channels)
            self.conv2 = DenseSAGEConv(hidden_channels, hidden_channels, normalize)
            self.bn2 = torch.nn.BatchNorm1d(hidden_channels)
            self.conv3 = DenseSAGEConv(hidden_channels, out_channels, normalize)
            self.bn3 = torch.nn.BatchNorm1d(out_channels)
            if lin is True:
                self.lin = torch.nn.Linear(
                    2 * hidden_channels + out_channels,
                    out_channels
                )
            else:
                self.lin = None

        def bn(self, i, x):
            batch_size, num_nodes, num_channels = x.size()
            x = x.view(-1, num_channels)
            x = getattr(self, 'bn{}'.format(i))(x)
            x = x.view(batch_size, num_nodes, num_channels)
            return x

        def forward(self, x, adj, mask=None):
            x0 = x
            if pooling == 'diffpool':
                x1 = self.bn(1, F.relu(self.conv1(x0, adj, mask)))
                x2 = self.bn(2, F.relu(self.conv2(x1, adj, mask)))
                x3 = self.bn(3, F.relu(self.conv3(x2, adj, mask)))
            elif pooling == 'mincut':
                x1 = F.relu(self.conv1(x0, adj, mask))
                x2 = F.relu(self.conv2(x1, adj, mask))
                x3 = F.relu(self.conv3(x2, adj, mask))
            x = torch.cat([x1, x2, x3], dim=-1)
            if self.lin is not None:
                x = F.relu(self.lin(x))
            return x

    #Net to compute pooling in an unsupervised manner
    class Net(torch.nn.Module):
        def __init__(self):
            super(Net, self).__init__()
            self.gnn1_pool = GNN(data.num_node_features, 64, num_clusters)
            self.gnn1_embed = GNN(data.num_node_features, 64, 64, lin=False)
            self.pooling = pooling_selector[pooling]

        def forward(self, x, adj, mask=None):
            s = self.gnn1_pool(x, adj, mask)
            x = self.gnn1_embed(x, adj, mask)
            x, adj, l1, e1 = self.pooling(x, adj, s, mask)
            return torch.softmax(s, dim=-1), l1, e1, adj # returns assignation matrix, and auxiliary losses and new adj matrix




    ###########
    #Training
    ###########
    #Optimizer, model
    model = Net().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    #Trains using auxiliary losses only
    def train(epoch):
        model.train()
        optimizer.zero_grad()
        s, l1, e1, adj = model(data.x, data.adj)
        loss = l1 + e1
        loss.backward()
        optimizer.step()
        return loss

    #Measures mutual information of clustering computed by pooling and ground truth
    @torch.no_grad()
    def test():
        model.eval()
        pred_node_label = model(data.x, data.adj)[0].max(dim=-1)[1].detach().cpu().numpy() 
        truth_node_labels = data.y.cpu().numpy()
        nmi = normalized_mutual_info_score(truth_node_labels.flatten(), pred_node_label.flatten())
        return nmi


    log_handle = open(osp.join(outdir, 'log.txt'), 'w')
    best_val_acc = best_test_acc = 0
    for epoch in range(1, 251):
        train_loss = train(epoch)
        val_acc = test()
        test_acc = test()
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_test_acc = test_acc
            save_best_model(model, optimizer, epoch, outdir)
        print(f'Epoch: {epoch:03d}, Train Loss: {train_loss:.8f}, '
              f'Val NMI: {val_acc:.4f}, Test NMI: {test_acc:.4f}')
        log_handle.write(f'Epoch: {epoch:03d}, Train Loss: {train_loss:.8f}, '
              f'Val NMI: {val_acc:.4f}, Test NMI: {test_acc:.4f}\n')

    print(f'Best Val NMI: {best_val_acc:.4f}, Test NMI: {best_test_acc:.4f}')
    log_handle.write(f'\nBest Val NMI: {best_val_acc:.4f}, Test NMI: {best_test_acc:.4f}\n')
    log_handle.close()



    #Loads best model
    model = load_best_model(model, outdir)
    model.eval()
    node_assignation = model(data.x, data.adj)[0].max(-1)[1].detach().cpu().numpy().squeeze()
    adj_matrix = model(data.x, data.adj)[3].detach().cpu().numpy().squeeze()


    #Computes node clustering and adj matrices and saves them
    pd.DataFrame(node_assignation).transpose().to_csv(osp.join(outdir, 'node_assignation.csv'), header = None, index=False)
    pd.DataFrame(adj_matrix).to_csv(osp.join(outdir, 'adj_matrix.csv'), header = None, index=False)





The obtained data WikiCS has 11701 nodes, 297110 edges, 300 features, 10 labels, 
Epoch: 001, Train Loss: 0.16943401, Val NMI: 0.0112, Test NMI: 0.0112
Epoch: 002, Train Loss: 0.16939914, Val NMI: 0.0114, Test NMI: 0.0114
Epoch: 003, Train Loss: 0.16936421, Val NMI: 0.0110, Test NMI: 0.0110
Epoch: 004, Train Loss: 0.16931725, Val NMI: 0.0121, Test NMI: 0.0121
Epoch: 005, Train Loss: 0.16924536, Val NMI: 0.0125, Test NMI: 0.0125
Epoch: 006, Train Loss: 0.16913784, Val NMI: 0.0129, Test NMI: 0.0129
Epoch: 007, Train Loss: 0.16898727, Val NMI: 0.0133, Test NMI: 0.0133
Epoch: 008, Train Loss: 0.16878963, Val NMI: 0.0135, Test NMI: 0.0135
Epoch: 009, Train Loss: 0.16853356, Val NMI: 0.0130, Test NMI: 0.0130
Epoch: 010, Train Loss: 0.16820109, Val NMI: 0.0132, Test NMI: 0.0132
Epoch: 011, Train Loss: 0.16775584, Val NMI: 0.0131, Test NMI: 0.0131
Epoch: 012, Train Loss: 0.16716516, Val NMI: 0.0134, Test NMI: 0.0134
Epoch: 013, Train Loss: 0.16640747, Val NMI: 0.0137, Test NMI: 0.0137
Epoch: 0

Epoch: 117, Train Loss: -0.12427604, Val NMI: 0.3239, Test NMI: 0.3239
Epoch: 118, Train Loss: -0.12573659, Val NMI: 0.3253, Test NMI: 0.3253
Epoch: 119, Train Loss: -0.12722379, Val NMI: 0.3269, Test NMI: 0.3269
Epoch: 120, Train Loss: -0.12854856, Val NMI: 0.3279, Test NMI: 0.3279
Epoch: 121, Train Loss: -0.12988949, Val NMI: 0.3311, Test NMI: 0.3311
Epoch: 122, Train Loss: -0.13130206, Val NMI: 0.3327, Test NMI: 0.3327
Epoch: 123, Train Loss: -0.13260776, Val NMI: 0.3347, Test NMI: 0.3347
Epoch: 124, Train Loss: -0.13385499, Val NMI: 0.3361, Test NMI: 0.3361
Epoch: 125, Train Loss: -0.13515508, Val NMI: 0.3379, Test NMI: 0.3379
Epoch: 126, Train Loss: -0.13641375, Val NMI: 0.3385, Test NMI: 0.3385
Epoch: 127, Train Loss: -0.13766330, Val NMI: 0.3407, Test NMI: 0.3407
Epoch: 128, Train Loss: -0.13894010, Val NMI: 0.3432, Test NMI: 0.3432
Epoch: 129, Train Loss: -0.14012182, Val NMI: 0.3450, Test NMI: 0.3450
Epoch: 130, Train Loss: -0.14134067, Val NMI: 0.3463, Test NMI: 0.3463
Epoch:

Epoch: 233, Train Loss: -0.20488006, Val NMI: 0.3783, Test NMI: 0.3783
Epoch: 234, Train Loss: -0.20503014, Val NMI: 0.3787, Test NMI: 0.3787
Epoch: 235, Train Loss: -0.20519340, Val NMI: 0.3773, Test NMI: 0.3773
Epoch: 236, Train Loss: -0.20538813, Val NMI: 0.3782, Test NMI: 0.3782
Epoch: 237, Train Loss: -0.20559502, Val NMI: 0.3773, Test NMI: 0.3773
Epoch: 238, Train Loss: -0.20579368, Val NMI: 0.3781, Test NMI: 0.3781
Epoch: 239, Train Loss: -0.20598394, Val NMI: 0.3783, Test NMI: 0.3783
Epoch: 240, Train Loss: -0.20616257, Val NMI: 0.3768, Test NMI: 0.3768
Epoch: 241, Train Loss: -0.20632154, Val NMI: 0.3778, Test NMI: 0.3778
Epoch: 242, Train Loss: -0.20647222, Val NMI: 0.3763, Test NMI: 0.3763
Epoch: 243, Train Loss: -0.20662522, Val NMI: 0.3783, Test NMI: 0.3783
Epoch: 244, Train Loss: -0.20677376, Val NMI: 0.3767, Test NMI: 0.3767
Epoch: 245, Train Loss: -0.20691735, Val NMI: 0.3784, Test NMI: 0.3784
Epoch: 246, Train Loss: -0.20705742, Val NMI: 0.3765, Test NMI: 0.3765
Epoch: